<a href="https://www.kaggle.com/code/amrmohamedsaber/cat-dog-classification?scriptVersionId=186960483" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Import Libraries**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, AveragePooling2D,Dropout
from keras.utils import to_categorical
from keras.optimizers import SGD
import matplotlib.pyplot as plt
import keras
from keras.applications.vgg16 import VGG16
from keras.applications import ResNet50,InceptionV3,DenseNet121
from keras import layers, models
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from keras.metrics import Precision,Recall,F1Score
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
import seaborn as sns
import matplotlib.pyplot as plt
import os
from glob import glob
import shutil
import random
from shutil import copyfile
from keras.preprocessing.image import ImageDataGenerator

# create files for train,test and validate

In [ ]:
try:
    os.mkdir('/kaggle/working/cats-v-dogs')
    os.mkdir('/kaggle/working/cats-v-dogs/training')
    os.mkdir('/kaggle/working/cats-v-dogs/validation')
    os.mkdir('/kaggle/working/cats-v-dogs/test')
    os.mkdir('/kaggle/working/cats-v-dogs/training/cats')
    os.mkdir('/kaggle/working/cats-v-dogs/training/dogs')
    os.mkdir('/kaggle/working/cats-v-dogs/validation/cats')
    os.mkdir('/kaggle/working/cats-v-dogs/validation/dogs')
    os.mkdir('/kaggle/working/cats-v-dogs/test/cats')
    os.mkdir('/kaggle/working/cats-v-dogs/test/dogs')
except OSError:
    print('Error failed to make directory')

In [ ]:
#Define data path
CAT_DIR = '/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat'
DOG_DIR = '/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog'

TRAINING_DIR = "/kaggle/working/cats-v-dogs/training/"
VALIDATION_DIR = "/kaggle/working/cats-v-dogs/validation/"

TRAINING_CATS = os.path.join(TRAINING_DIR, "cats/")
VALIDATION_CATS = os.path.join(VALIDATION_DIR, "cats/")

TRAINING_DOGS = os.path.join(TRAINING_DIR, "dogs/")
VALIDATION_DOGS = os.path.join(VALIDATION_DIR, "dogs/")

INCLUDE_TEST = True

# Function to split and images

In [ ]:
def split_data(main_dir, training_dir, validation_dir, test_dir=None, include_test_split=True, split_size=0.8, random_seed=42):
    # Get the list of all files in the main directory
    all_files = [file for file in os.listdir(main_dir) if os.path.getsize(os.path.join(main_dir, file)) > 0]

    # Split the data into training and (validation + test) sets
    train_files, remaining_files = train_test_split(all_files, train_size=split_size, random_state=random_seed)

    # Split the remaining data into validation and test sets if include_test_split is True
    if include_test_split:
        validation_files, test_files = train_test_split(remaining_files, test_size=0.5, random_state=random_seed)
    else:
        validation_files, test_files = remaining_files, []

    # Copy files to their respective directories
    copy_files(main_dir, training_dir, train_files)
    copy_files(main_dir, validation_dir, validation_files)
    if include_test_split:
        copy_files(main_dir, test_dir, test_files)

    print("Split successful!")

def copy_files(source_dir, destination_dir, file_list):
    # Create destination directory if it doesn't exist
    os.makedirs(destination_dir, exist_ok=True)

    # Copy files from source to destination
    for file in file_list:
        source_path = os.path.join(source_dir, file)
        destination_path = os.path.join(destination_dir, file)
        shutil.copyfile(source_path, destination_path)

In [ ]:
split_data(CAT_DIR, '/kaggle/working/cats-v-dogs/training/cats', '/kaggle/working/cats-v-dogs/validation/cats', '/kaggle/working/cats-v-dogs/test/cats',INCLUDE_TEST, 0.8)
split_data(DOG_DIR, '/kaggle/working/cats-v-dogs/training/dogs', '/kaggle/working/cats-v-dogs/validation/dogs','/kaggle/working/cats-v-dogs/test/dogs',INCLUDE_TEST, 0.8)

# Preprocess Data

In [ ]:
train_gen = ImageDataGenerator(
        rescale=1./255)


validation_gen =  ImageDataGenerator(
        rescale=1./255.)

if INCLUDE_TEST:
    test_gen =  ImageDataGenerator(
            rescale=1./255.)
train_generator = train_gen.flow_from_directory(
        '/kaggle/working/cats-v-dogs/training',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')
validation_generator = validation_gen.flow_from_directory(
        '/kaggle/working/cats-v-dogs/validation',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

if INCLUDE_TEST:
    test_generator = test_gen.flow_from_directory(
        '/kaggle/working/cats-v-dogs/test',
        target_size=(150, 150),
        batch_size=32,
        shuffle= False,
        class_mode='binary')

In [ ]:
print(len(os.listdir('/kaggle/working/cats-v-dogs/training/cats')))
print(len(os.listdir('/kaggle/working/cats-v-dogs/training/dogs')))

print(len(os.listdir('/kaggle/working/cats-v-dogs/validation/cats')))
print(len(os.listdir('/kaggle/working/cats-v-dogs/validation/dogs')))


print(len(os.listdir('/kaggle/working/cats-v-dogs/test/cats')))
print(len(os.listdir('/kaggle/working/cats-v-dogs/test/dogs')))

# CNN Model

In [ ]:
model =Sequential ()
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='valid',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics = ['accuracy'])
result = model.fit(
        train_generator,
        epochs=20,
        validation_data=validation_generator)

In [ ]:
if INCLUDE_TEST:
    score, acc = model.evaluate(test_generator)
    print('Test lose:', score)
    print('Test accuracy:', acc)

In [ ]:
plt.plot(result.history['accuracy'],color='red',label='train')
plt.plot(result.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(result.history['loss'],color='red',label='train')
plt.plot(result.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

# CNN Model After Solve Overfitting

In [ ]:
model2 = Sequential()

model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model2.add(layers.BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.2))

# Block 1
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.2))

# Block 2
model2.add(Conv2D(128, (3, 3), activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.2))

# Block 3
model2.add(Conv2D(256, (3, 3), activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.2))

# Fully Connected layers
model2.add(Flatten())
model2.add(Dense(1024, activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(Dropout(0.2))

# Output layer
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

**Methods for solving Overfitting**

In [ ]:
train_gen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        channel_shift_range=20)
train_generator = train_gen.flow_from_directory(
        '/kaggle/working/cats-v-dogs/training',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

early_stoping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [ ]:

model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(),Recall()])

result2 = model2.fit(
        train_generator,
        callbacks=[early_stoping,learning_rate_reduction],
        epochs=20,
        validation_data=validation_generator,
        shuffle=True)

In [ ]:
if INCLUDE_TEST:
    score, acc, pre, rcl = model2.evaluate(test_generator)
    f1=2*((pre*rcl)/(pre+rcl))
    print("===============")
    print('Test Loss:', score)
    print('Test Accuracy:', acc)
    print('Test Precision:',pre)
    print('Test Recall:',rcl)
    print('Test F1-Score:',f1)
    print("===============")
    print("===============")
    # Generate predictions for the test set using the evaluation generator
    predictions = model2.predict(test_generator)

    # Convert probabilities to binary predictions
    binary_predictions = np.round(predictions)

    # Get true labels
    true_labels = test_generator.classes

    # Calculate confusion matrix
    conf_matrix = confusion_matrix(true_labels, binary_predictions)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Cat', 'Dog'], yticklabels=['Cat', 'Dog'])
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()


In [ ]:
plt.plot(result2.history['accuracy'],color='red',label='train')
plt.plot(result2.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(result2.history['loss'],color='red',label='train')
plt.plot(result2.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
class_names = ['Cat', 'Dog']
def plot_prediction(generator, n_images):
    images, labels = generator.next()
    
    # Gets the model predictions
    preds = model2.predict(images)
    threshold = 0.5
    predictions = (preds > threshold).astype(int)
    labels = labels.astype('int32')
    
    plt.figure(figsize=(14, 15))
    
    for i, (image, label, pred) in enumerate(zip(images, labels, preds), 1):
        plt.subplot(4, 3, i)
        plt.imshow(image)
        
        title_color = 'g' if predictions[i - 1] == label else 'r'
        title = f"{class_names[label]}: "
        title_obj = plt.title(title)
        plt.setp(title_obj, color=title_color) 
        
        plt.axis('off')
        
        if i == n_images:
            break
    
    plt.show()

In [ ]:
if INCLUDE_TEST:
    plot_prediction(test_generator, 9)

# DenseNet121 Model 

In [ ]:
base_model  = DenseNet121(weights='imagenet',
                    include_top=False,
                    input_shape=(150,150,3)
                   )
base_model = models.Model(inputs=base_model.inputs, outputs=base_model.output)

model4 = models.Sequential()
model4.add(base_model)
model4.add(layers.Flatten())
model4.add(layers.BatchNormalization())
model4.add(layers.Dense(2048, activation='relu'))
model4.add(layers.Dropout(0.2))
model4.add(layers.Dense(1, activation='sigmoid'))

for layer in base_model.layers:
    layer.trainable = False


model4.summary()

In [ ]:
model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',Precision(),Recall()])
result4 = model4.fit(
        train_generator,
        epochs=20,
        callbacks=[early_stoping,learning_rate_reduction],
        validation_data=validation_generator,
        shuffle=True)

In [ ]:
if INCLUDE_TEST:
    score, acc, pre, rcl  = model4.evaluate(test_generator)
    f1=2*((pre*rcl)/(pre+rcl))
    print('Test Loss:', score)
    print('Test Accuracy:', acc)
    print('Test Precision:',pre)
    print('Test Recall:',rcl)
    print('Test F1-Score:',f1)
    # Generate predictions for the test set using the evaluation generator
    predictions = model4.predict(test_generator)

    # Convert probabilities to binary predictions
    binary_predictions = np.round(predictions)

    # Get true labels
    true_labels = test_generator.classes

    # Calculate confusion matrix
    conf_matrix = confusion_matrix(true_labels, binary_predictions)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Cat', 'Dog'], yticklabels=['Cat', 'Dog'])
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [ ]:
plt.plot(result4.history['accuracy'],color='red',label='train')
plt.plot(result4.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(result4.history['loss'],color='red',label='train')
plt.plot(result4.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
class_names = ['Cat', 'Dog']

def plot_prediction_with_prob(generator, n_images):
    # Get the images and the labels from the generator
    images, labels = generator.next()
    
    # Gets the model predictions
    preds = model4.predict(images)
    threshold = 0.5
    predictions = (preds > threshold).astype(int)
    labels = labels.astype('int32')
    
    plt.figure(figsize=(14, 15))
    
    for i, (image, label, pred) in enumerate(zip(images, labels, preds), 1):
        plt.subplot(4, 3, i)
        plt.imshow(image)
        
        title_color = 'g' if predictions[i - 1] == label else 'r'
        title = f"{class_names[label]}: "
        title_obj = plt.title(title)
        plt.setp(title_obj, color=title_color) 
        
        plt.axis('off')
        
        if i == n_images:
            break

In [ ]:
if INCLUDE_TEST:
    plot_prediction_with_prob(test_generator, 9)